# SHP data preparation

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import random
import cv2
import os
from scipy.stats import ttest_ind
from scipy import stats
from scipy.stats import pearsonr
from skimage import io
from pyproj import Geod
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\bahr\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\bahr\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


This file prepare the SHP survey data for further analysis.

In [1]:
# Set up dir

dir_h = "YOUR_DIRECTORY/"

## SHP W23 September 2021 - March 2022

In [3]:
shp21_p = pd.read_stata(dir_h + "shp21_p_user.dta", preserve_dtypes=False, convert_categoricals=False)
shp21_h = pd.read_stata(dir_h + "shp21_h_user.dta", preserve_dtypes=False, convert_categoricals=False)
shp21_geo = pd.read_csv(dir_h + "shpgeo21.csv")
ch_postcode_population = pd.read_excel(dir_h + "population_postcode_CH.xlsx")

C:\Users\bahr\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### Merge geo and population information to household data

Remove duplicate household ids

In [4]:
shp21_geo = shp21_geo.drop_duplicates(subset="idhous21")
len(shp21_geo)

8422

Merge household with geo data

In [5]:
shp21_h = shp21_h.merge(shp21_geo, how="left", on="idhous21")

Merge household with geo and population data

In [6]:
shp21_h = shp21_h.merge(ch_postcode_population, left_on="npa", right_on="postcode", how="left")

### Merge household with individual data

In [7]:
shp21 = shp21_p.merge(shp21_h, how="left", on="idhous21")

### Generate individual level data set 

In [8]:
shp21 = shp21[["idpers", "idhous21", "npa", "g_lat", "g_lon", "population",
               "pdate21", "i21eqon", "i21ptotn", "ownkid21", "nbkid21", "age21",
              "p21c01", "p21c02", "civsta21", "wstat21", "edyear21",
              "p21w39", "p21w42", "p21w34a", "occupa21", "sex21", "h21h27", 
               "h21h28","p21c44", "com2_21", "h21h15", "h21i02", "h21h37",
               "h21h12", "hldtyp21", "h21h29", "educat21", "p21c17", "p21c18",
              "p21c184", "p21a01", "p21a04", "p21ql04"]]

In [9]:
shp21["n_hhmembers_18_14"] = shp21.groupby("idhous21").age21.transform(lambda x: np.sum(np.where(x < 18,1,0)))
shp21["n_hhmembers_o14"] = shp21.groupby("idhous21").age21.transform(lambda x: np.sum(np.where(x > 14,1,0)))
shp21["n_hhmembers_u14"] = shp21.groupby("idhous21").age21.transform(lambda x: np.sum(np.where(x <= 14,1,0)))
shp21["n_hhmembers_u14"] = np.where(shp21.nbkid21 > shp21.n_hhmembers_18_14, shp21.n_hhmembers_u14 + (shp21.nbkid21-shp21.n_hhmembers_18_14), shp21.n_hhmembers_u14)

shp21["CH_region"] = np.where(shp21.npa < 3000, 0,
                             np.where((shp21.npa >= 6000) & (shp21.npa < 7000), 1, 2))
shp21["workload"] = np.where(shp21.p21w39==2, 100, 
                             np.where(~shp21.occupa21.isin([1, 2, 3]), 0, shp21.p21w42))
shp21["position"] = np.where(~shp21.occupa21.isin([1, 2, 3]), 5, shp21.p21w34a) # not in work force is recoded to 5
shp21["pers_income"] = np.where(shp21.occupa21.isin([4, 6, 7, 11]), 0, shp21.i21ptotn)

Set negative values to missings

In [10]:
hh_type_map = {1:0,
               2:0,
               3:0,
               4:1,
               5:1,
               6:0,
               7:0,
               8:1,
               9:1,
               10:1,
               11:1,
               12:0,
               13:0}
shp21 = shp21.assign(hh_type = shp21.hldtyp21.map(hh_type_map))

In [11]:
shp21["hh_eq_income"] = np.where(shp21.i21eqon < 0, np.NAN, shp21.i21eqon)
shp21["pers_income"] = np.where(shp21.pers_income == -4, 0,
                                np.where(shp21.pers_income.isin([-8, -3, -2, -1]), np.NAN, shp21.pers_income))
shp21["nchildren"] = np.where(shp21.ownkid21 < 0, 0, shp21.ownkid21)
shp21["age"] = np.where(shp21.age21 < 0, np.NAN, shp21.age21) 
shp21["health"] = np.where(shp21.p21c01 < 1, np.NAN, shp21.p21c01)
shp21["healthsat"] = np.where(shp21.p21c02 < 0, np.NAN, shp21.p21c02)
shp21["civstat"] = np.where(shp21.civsta21 < 1, np.NAN, shp21.civsta21)
shp21["wstat"] = np.where(shp21.wstat21 < 0, np.NAN, shp21.wstat21) 
shp21["edyear"] = np.where(shp21.edyear21 < 0, np.NAN, shp21.edyear21) 
shp21["educat"] = np.where(shp21.educat21 < 0, np.NAN, shp21.educat21) 
shp21["workload"] = np.where(shp21.workload < 0, -1, shp21.workload)  # impute missings with -1 and generate missing dummy
shp21["workload_mis"] = np.where(shp21.workload < 0, 1, 0)
shp21["position"] = np.where(shp21.position < 1, -1, shp21.position) # impute missings with -1 and generate missing dummy
shp21["position_mis"] = np.where(shp21.position < 1, 1, 0)
shp21["occupa"] = np.where(shp21.occupa21 < 1, -1, shp21.occupa21) # impute missings with -1 and generate missing dummy
shp21["occupa_mis"] = np.where(shp21.occupa < 1, 1, 0)
shp21["sex"] = np.where(shp21.sex21==2, 1, 0)
shp21["prob_env"] = np.where(shp21.h21h27 < 1, np.NAN, shp21.h21h27) 
shp21["prob_crime"] = np.where(shp21.h21h28 < 1, np.NAN, shp21.h21h28) 
shp21["community_type"] = np.where(shp21.com2_21 < 1, np.NAN, shp21.com2_21) 
shp21["residence_type"] = np.where(shp21.h21h15 < 1, np.NAN, shp21.h21h15)  
shp21["garden_terrace"] = np.where(shp21.h21i02 < 1, np.NAN, shp21.h21i02)
shp21["accommodation_exp"] = np.where(((shp21.h21h37 < 1) | (shp21.h21h37 > 10000)), np.NAN, shp21.h21h37)
shp21["accommodation_exp_log"] = np.log(shp21.accommodation_exp+1)
shp21["accommodation_sat"] = np.where(shp21.h21h12 < 1, np.NAN, shp21.h21h12)
shp21["owner"] = np.where(shp21.h21h29 == 2, 1, 
                          np.where(shp21.h21h29.isin([1, 3]), 0, np.NAN))
shp21["lifesat"] = np.where(shp21.p21c44 < 0, np.NAN, shp21.p21c44) 

shp21["n_hhmembers_u5"] = shp21.groupby("idhous21").age21.transform(lambda x: np.sum(np.where(x <= 4,1,0)))
shp21["n_hhmembers_17_5"] = shp21.groupby("idhous21").age21.transform(lambda x: np.sum(np.where(((x > 4) & (x < 18)),1,0)))
shp21["nchildren_hh"] = shp21.groupby("idhous21").nchildren.transform("max")
shp21["nchildren"] = np.where(((shp21.ownkid21 < 0) & (shp21.age < 25)), 0,
                              np.where(((shp21.ownkid21 < 0) & (shp21.age > 25) & (shp21.hh_type==1)), shp21.nchildren_hh, 
                                      np.where(((shp21.ownkid21 < 0) & (shp21.age > 25) & (shp21.hh_type==0)), 0, 
                                               np.where(shp21.ownkid21 >=0, shp21.ownkid21, np.NaN))))

shp21["n_children_u5"] = np.where(shp21.nchildren > 0, shp21.n_hhmembers_u5, 0)
shp21["n_children_17_5"] = np.where(shp21.nchildren > 0, shp21.n_hhmembers_17_5, 0)
shp21["nchildren"] = np.where(shp21.nchildren < shp21.n_children_u5 + shp21.n_children_17_5, shp21.n_children_u5 + shp21.n_children_17_5, shp21.nchildren)
shp21["n_children_o17"] = shp21.nchildren - shp21.n_children_u5 - shp21.n_children_17_5
shp21["depression"] = np.where(shp21.p21c17 < 0, np.NAN, shp21.p21c17)
shp21["optimism"] = np.where(shp21.p21c18 < 0, np.NAN, shp21.p21c18)
shp21["stress"] = np.where(shp21.p21c184 < 0, np.NAN, shp21.p21c184)
shp21["relationship_sat"] = np.where(shp21.p21ql04 < 0, np.NAN, shp21.p21ql04)
shp21["sport"] = np.where(shp21.p21a01 == 2, 0,
                         np.where((shp21.p21a01 == 1) & (shp21.p21a04 > 0), shp21.p21a04, np.NAN))


Save variables for income imputation

In [12]:
income_imp = shp21[["idpers", "pers_income", "age", "edyear", "sex", "occupa", "occupa_mis", "workload", "workload_mis","position", "position_mis", "CH_region", "g_lat", "g_lon"]]
income_imp.to_pickle(dir_h + "income_imp.pkl")

=> proceede with file ```pers_income_imputation.ipynb```

Load imputed income data and add it to existing data.

In [13]:
income_imputed = pd.read_pickle(dir_h + "income_imputed.pkl")
shp21 = shp21.merge(income_imputed, how="left", on="idpers")

Calculate household equivalent income by the modified OECD scale.

In [14]:
def oecd_hhmembers(data_1, data_2):
    if data_1 > 1:
        hhmembers = 1 + ((data_1-1)*0.5) + (data_2*0.3)
    elif data_1 == 1:
        hhmembers = 1 + (data_2*0.3)
    else:
        raise Warning("does not work")
        
    return hhmembers

In [15]:
shp21 = shp21.assign(oecd_hhmembers = shp21.apply(lambda x: oecd_hhmembers(x.n_hhmembers_o14, x.n_hhmembers_u14), axis=1))
shp21 = shp21.assign(hhincome = shp21.groupby("idhous21").pers_income_imp.transform("sum"))
shp21 = shp21.assign(hh_eq_income_imp = shp21.hhincome/shp21.oecd_hhmembers)
shp21 = shp21.assign(hh_eq_income_log = np.log(shp21.hh_eq_income+1))
shp21 = shp21.assign(hh_eq_income_imp_log = np.log(shp21.hh_eq_income_imp+1))

Correlation and mean deviation between imputed houshold equivalent income and houshold equivalent income provided by FORS.

In [16]:
print(pearsonr(shp21[~shp21.hh_eq_income.isna()].hh_eq_income_imp, shp21[~shp21.hh_eq_income.isna()].hh_eq_income))
np.mean(abs(shp21[~shp21.hh_eq_income.isna()].hh_eq_income_imp-shp21[~shp21.hh_eq_income.isna()].hh_eq_income))

(0.9185040608814452, 0.0)


8171.983404707293

#### Recode health, occupation, civil status, and problems with environment or criminality

In [17]:
health_map = {1:5,
             2:4,
             3:3,
             4:2,
             5:1}
shp21 = shp21.assign(health = shp21.health.map(health_map))

In [18]:
occupa_map = {1:1, # full-time
             2:2, # part-time
             3:2, # part-time
             4:3, # education
             10:4, # unemployed
             8:5, # retired
             9:5, # retired
             5:6, # other occupation
             6:6, # other occupation
             7:6, # other occupation
             11:6} # other occupation
shp21 = shp21.assign(occupa = shp21.occupa.map(occupa_map))

In [19]:
civ_map = {1:1, # single
          2:2, # married or partnership
          6:2, # married or partnership
          3:3, # separated, divorced, dissolved, windowed
          4:3, # separated, divorced, dissolved, windowed
          5:3, # separated, divorced, dissolved, windowed
          7:3} # separated, divorced, dissolved, windowed
shp21 = shp21.assign(civstat = shp21.civstat.map(civ_map))

#### Add cities to postcodes

In [20]:
postcode_to_city = pd.read_excel(dir_h + "postcode_to_city.xlsx")

In [21]:
postcode_to_city = postcode_to_city[~postcode_to_city.npa.duplicated(keep="first")]
city_code = pd.DataFrame(postcode_to_city.city.unique(), columns=["city"])
city_code.insert(1, "city_code", range(1, 1+len(city_code)))
postcode_to_city = postcode_to_city.merge(city_code, on="city", how="left")
shp21 = shp21.merge(postcode_to_city, on="npa", how="left")

#### Add population density

In [22]:
CH_shp = gpd.read_file(dir_h + "switzerland_map/PLZO_SHP_LV95/PLZO_PLZ.shp")
CH_shp = CH_shp[["PLZ", "geometry"]]
CH_shp["geometry"] = CH_shp.geometry.to_crs({'init': 'epsg:2056'})
CH_shp["area"] = CH_shp.geometry.area/10**6
CH_shp = CH_shp[["PLZ", "area"]]
CH_shp["area"] = CH_shp.groupby("PLZ").area.transform("sum")
CH_shp = CH_shp.drop_duplicates()

C:\Users\bahr\Anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [23]:
ch_postcode_population = pd.read_excel(dir_h + "population_postcode_CH.xlsx")
CH_shp = CH_shp.merge(ch_postcode_population, left_on="PLZ", right_on="postcode", how="left")
CH_shp["pop_density"] = CH_shp["population"]/CH_shp["area"]
CH_shp[["PLZ", "area", "pop_density"]].to_pickle(dir_h + "postcode_area.pkl")

C:\Users\bahr\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [24]:
postcode_area = pd.read_pickle(dir_h + "postcode_area.pkl")
shp21 = shp21.merge(postcode_area, left_on="npa", right_on="PLZ", how="left")

#### Add income per capita on municipality level

In [25]:
shp21["city"] = shp21["city"].replace({"Petit-Lancy": "Lancy",
                                              "Grand-Lancy": "Lancy",
                                              "Corin-de-la-Crête": "Crans-Montana",
                                              "Bussigny-près-Lausanne": "Bussigny",
                                              "Glattpark (Opfikon)": "Opfikon",
                                              "Hermatswil" : "Pfäffikon",
                                              "Gockhausen": "Dübendorf",
                                              "Ottikon b. Kemptthal": "Illnau-Effretikon",
                                              "Reutlingen (Winterthur)": "Winterthur",
                                              "Siebnen": "Wangen SZ",
                                              "Jona": "Rapperswil-Jona",
                                              "Le Lignon": "Vernier",
                                              "Les Acacias": "Genève",
                                              "Emmenbrücke": "Emmen",
                                              "Appenzell Eggerstanden": "Schwende",
                                              "Wilen b. Wollerau": "Freienbach",
                                              "Frasnacht": "Arbon",
                                               "Buonas": "Risch",
                                               "Sundlauenen": "Beatenberg",
                                               "Trachslau": "Einsiedeln",
                                               "Deisswil b. Münchenbuchsee": "Deisswil bei Münchenbuchsee",
                                               "Lüchingen": "Altstätten",
                                               "Brugg AG": "Brugg"})

In [26]:
income_municipality = pd.read_excel(dir_h + "income_by_municipality.xlsx")
income_municipality = income_municipality.rename(columns={"GEO_NAME": "city",
                                                         "VALUE": "mun_income_per_cap"})
income_municipality = income_municipality[income_municipality.VARIABLE=="mun_income_per_capita"][["city", "mun_income_per_cap"]]
shp21 = shp21.merge(income_municipality, how="left", on="city")
shp21["mun_income_per_cap_log"] = np.log(shp21.mun_income_per_cap)

#### Add location of next larger train station as proxy for city center

In [27]:
# remove wrongly coded locations and change the city of one household to Zurich
shp21 = shp21[~shp21.idhous21.isin([646502, 100603, 129723, 222851])]
shp21_changed = shp21[shp21.idhous21==423931].replace({"Hinteregg": "Zürich", 8132: 8008})
shp21 = pd.concat([shp21[shp21.idhous21!=423931], shp21_changed]).reset_index(drop=True)

In [28]:
train_station_location = pd.read_excel(dir_h + "/train_station_location.xlsx")
train_station_location["center_g_lat"] = train_station_location.center_coord.apply(lambda x: x.split(',')[0])
train_station_location["center_g_lon"] = train_station_location.center_coord.apply(lambda x: x.split(',')[1])
shp21 = shp21.merge(train_station_location[["city","center_name", "center_g_lat", "center_g_lon"]], how="left", on="city")

In [29]:
coords_center = gpd.GeoDataFrame(shp21["idhous21"], geometry=gpd.points_from_xy(shp21.center_g_lat, shp21.center_g_lon), crs=4326)
coords_center = coords_center.to_crs(crs=21782)
coords = gpd.GeoDataFrame(shp21["idhous21"], geometry=gpd.points_from_xy(shp21.g_lat, shp21.g_lon), crs=4326)
coords = coords.to_crs(crs=21782)
shp21["distance_to_center_1"] = coords.distance(coords_center)

In [30]:
g = Geod(ellps='WGS84')
distance = []

for i in range(0, len(shp21)):
    df = shp21.iloc[i]
    _, _, dist = g.inv(df.g_lat, df.g_lon, df.center_g_lat, df.center_g_lon)
    distance.append(dist)

In [31]:
shp21["distance_to_center_2"] = distance

#### Restricte sample to observations between 20 and 85 years

4333 observations are removed

In [32]:
shp21 = shp21[(shp21.age >= 20) & (shp21.age <= 85)]

#### Remove observations from ural postcodes (less than 500 inhabtiants per km2)

6481 observations are removed

In [33]:
shp21 = shp21[shp21.pop_density >= 500]

#### Remove obervations with missings on exogenous variable (life satisfaction)

2243 of the observations have not indicated their life satisfaction.

In [34]:
shp21_removed = shp21[shp21.lifesat.isna()]
print(len(shp21_removed))
shp21 = shp21[~shp21.lifesat.isna()]
print(len(shp21))

2243
6918


In [35]:
shp21.to_pickle(dir_h + "shp21_preliminar.pkl")

#### Examine possible seasonal effect

Because interviews are conducted from September 2021 until March 2022 it might be that life satisfaction differs between fall and winter interviewees. Further the effect of the neighborhood topology and especially greenery on life satisfaction might be weeker during the winter. Therefore a seperate analysis will be conducted for the interviews until 31th of October. Further, the reports on the weather conditions of fall 2021 from the Federal Office of Meteorology and Climatology indicate a warm and sunny fall and the first snowfall was registered at the 1st November [report](https://www.meteoschweiz.admin.ch/service-und-publikationen/publikationen.html#order=date-desc&page=2&pageGroup=publication&type=reportOrBulletin). 

3970 households were interviewed until the 1th of november or after the 1th of march and 4873 households in the winter season in between. If focussing at the urban population 1467 households were interviewed in fall or spring and 1829 in winter. We assume that the interview date was assigned at random and the interviewees were therefore not able to select themself into a season. If this holds true we can conclude that the season has no effect on life satisfaction since mean, median and standard deviation are not differing between the seasons. A two-sample T-test support this assumption. Further, no significant difference between the two sample can be found for the variables education years and household equivalence income. However, for number of children per household and age the sample means differ from each other significantly.

In [39]:
def ttest(s1, s2):
    diff_means = s1.mean() - s1.mean()
    t_stat, p_value = ttest_ind(s1, s2)
    print("Diff means:", diff_means)
    print("T-statistic value:", t_stat)
    print("P-Value:", p_value)

In [36]:
shp21["winter"] = np.where((shp21.pdate21 < "2021-11-01") | (shp21.pdate21 >= "2022-03-01"), 0, 1)

In [41]:
ttest(shp21[shp21.winter==0].lifesat, shp21[shp21.winter==1].lifesat)

Diff means: 0.0
T-statistic value: 0.6253294661764512
P-Value: 0.5317754768508031


In [40]:
ttest(shp21[(shp21.winter==0) & (shp21.pop_density >= 1500)].lifesat, shp21[(shp21.winter==1) & (shp21.pop_density >= 1500)].lifesat)

Diff means: 0.0
T-statistic value: 0.5291720200619229
P-Value: 0.5967221422838016


Final check of amount of missing values

In [38]:
shp21[["idpers", "idhous21", "npa", "g_lat", "g_lon", "pop_density", "CH_region", "city", "center_name",
       "distance_to_center_1", "distance_to_center_2", "prob_env", "sport", "depression",
      "lifesat", "sex", "age", "civstat", "educat", "occupa", "nchildren", "n_children_u5",
       "n_children_17_5", "n_children_o17", "hh_eq_income_log", "hh_eq_income_imp_log", "owner", "residence_type"]].isna().sum()

idpers                    0
idhous21                  0
npa                       0
g_lat                     0
g_lon                     0
pop_density               0
CH_region                 0
city                      0
center_name               0
distance_to_center_1      0
distance_to_center_2      0
prob_env                 52
sport                   127
depression               36
lifesat                   0
sex                       0
age                       0
civstat                   0
educat                   13
occupa                   23
nchildren                 1
n_children_u5             0
n_children_17_5           0
n_children_o17            1
hh_eq_income_log        756
hh_eq_income_imp_log      0
owner                    36
residence_type          290
dtype: int64

### Get nine tiles around center location

In [4]:
shp21_agg = shp21.groupby("idhous21").first()[["g_lat", "g_lon"]].reset_index()

In [5]:
def get_new_lat_lon(old_lat, old_lon, dy, dx, r_earth = 6378.137):
    pi = math.pi
    new_latitude = old_lat + (dy / r_earth) * (180 / pi)
    new_longitude = old_lon + ((dx / r_earth) * (180 / pi) / math.cos(new_latitude * pi/180))
    return new_latitude, new_longitude

In [6]:
new_coords=[]

for ids in shp21_agg.idhous21.tolist():
    old_latitude = shp21_agg[shp21_agg.idhous21==ids].g_lat.values[0]
    old_longitude = shp21_agg[shp21_agg.idhous21==ids].g_lon.values[0]

    hids=[]
    lats=[]
    lons=[]
    coords=[]
    tile_n=[]
    tile_iter=1

    for i in [[0.42, -0.42], [0.42, 0], [0.42, 0.42], [0, -0.42], [0, 0], [0, 0.42], [-0.42, -0.42], [-0.42, 0], [-0.42, 0.42]]:
        lat, lon = get_new_lat_lon(old_latitude, old_longitude, i[0], i[1])
        hids.append(ids)
        lats.append(lat)
        lons.append(lon)
        tile_n.append(tile_iter)
        tile_iter+=1
    coords.append(list(zip(hids, tile_n, lats, lons,)))
    new_coords.append(coords)

In [7]:
coords_array = np.array(new_coords)
np.save(dir_h + "/coords_array", coords_array)

=> proceed with file ```image_scraping_google.ipynb```